**Julia Kruk**

__Rozwiązanie pracy domowej nr 5__

## 1. Przygotowanie danych

1. Wykonaj `import` potrzebnych pakietów (oprócz poniżej wymienionych), tak aby umieszczony w tym notebooku kod działał.

In [1]:
import time
import sqlite3
import pickle
import pandas as pd

2. Wczytaj ramki danych, na których będziesz dalej pracował

**UWAGA:**
* Pliki muszą znajdować się w katalogu "travel_stackexchange_com" w tym samym katalogu co ten notebook (plik .ipynb).
* Nazwy tabel muszą być *zgodne z instrukcją zamieszczoną w treści pracy domowej*.


In [2]:
# DOPISZ TU ODPOWIEDNI KOD
Comments = pd.read_csv('./travel_stackexchange_com/Comments.csv')
Posts = pd.read_csv('./travel_stackexchange_com/Posts.csv')
Users = pd.read_csv('./travel_stackexchange_com/Users.csv')

3. Przygotuj bazę danych wykonując poniższą komórkę.

In [3]:
# NIE MODYFIKUJ TEJ KOMÓRKI
SCIEZKA_BAZY = './pd5_baza.db'  # Ścieżka do pliku z bazą danych ('./' oznacza bieżący katalog, czyli będzie to plik w tym samym katalogu, co ten notebook).
with sqlite3.connect(SCIEZKA_BAZY) as conn: # połączenie do bazy danych
    # wewnątrz bloku `with` mamy dostępny obiekt połączenia, które jest automatycznie zamykane po jego opuszczeniu.
    Comments.to_sql("Comments", conn, if_exists='replace')  # jeżeli ramka danych już istnieje, to jest nadpisywana.
    Posts.to_sql("Posts", conn, if_exists='replace')
    Users.to_sql("Users", conn, if_exists='replace')

## 2. Wyniki zapytań SQL

Wykonaj zapytania sql. Poniższy kod zapisze też wynik do pliku bazy - potem można go z niej odczytać.

**Uwaga**: Zapytania powinny się wykonywać nie dłużej niż kilka sekund każde,
jednak czasem występują problemy zależne od systemu, np. pod Linuxem zapytania 3 i 5 potrafią zająć
odp. kilka minut i ponad godzinę. Żeby obejść ten problem pod koniec tej sekcji pokazane jest, jak
zapisać wyniki zapytań do tymczasowych plików `pickle`.


In [9]:
# NIE MODYFIKUJ TEJ KOMÓRKI (poza ew. zakomentowaniem wywoływania zapytań)
zapytanie_1 = """
SELECT Location, SUM(UpVotes) as TotalUpVotes
FROM Users
WHERE Location != ''
GROUP BY Location
ORDER BY TotalUpVotes DESC
LIMIT 10
"""

zapytanie_2 = """
SELECT STRFTIME('%Y', CreationDate) AS Year,
       STRFTIME('%m', CreationDate) AS Month,
       COUNT(*) AS PostsNumber, MAX(Score) AS MaxScore
FROM Posts
WHERE PostTypeId IN (1, 2)
GROUP BY Year, Month
HAVING PostsNumber > 1000
"""

zapytanie_3 = """
SELECT Id, DisplayName, TotalViews
FROM (
        SELECT OwnerUserId, SUM(ViewCount) as TotalViews
        FROM Posts
        WHERE PostTypeId = 1
        GROUP BY OwnerUserId
    ) AS Questions
JOIN Users
ON Users.Id = Questions.OwnerUserId
ORDER BY TotalViews DESC
LIMIT 10
"""

zapytanie_4 = """
SELECT DisplayName, QuestionsNumber, AnswersNumber, Location, Reputation, UpVotes, DownVotes
FROM (
        SELECT *
        FROM (
                SELECT COUNT(*) as AnswersNumber, OwnerUserId
                FROM Posts
                WHERE PostTypeId = 2
                GROUP BY OwnerUserId
            ) AS Answers
        JOIN
            (
                SELECT COUNT(*) as QuestionsNumber, OwnerUserId
                FROM Posts
                WHERE PostTypeId = 1
                GROUP BY OwnerUserId
            ) AS Questions
        ON Answers.OwnerUserId = Questions.OwnerUserId
        WHERE AnswersNumber > QuestionsNumber
        ORDER BY AnswersNumber DESC
        LIMIT 5
    ) AS PostsCounts
JOIN Users
ON PostsCounts.OwnerUserId = Users.Id
"""

zapytanie_5 = """
SELECT Title, CommentCount, ViewCount, CommentsTotalScore, DisplayName, Reputation, Location
FROM (
        SELECT Posts.OwnerUserId, Posts.Title, Posts.CommentCount, Posts.ViewCount,
               CmtTotScr.CommentsTotalScore
        FROM (
                SELECT PostId, SUM(Score) AS CommentsTotalScore
                FROM Comments
                GROUP BY PostId
            ) AS CmtTotScr
        JOIN Posts ON Posts.Id = CmtTotScr.PostId
        WHERE Posts.PostTypeId=1
    ) AS PostsBestComments
JOIN Users ON PostsBestComments.OwnerUserId = Users.Id
ORDER BY CommentsTotalScore DESC
LIMIT 10
"""

def wykonaj_zapytanie(zap, k, conn_):
    """
    Wykonuje zapytanie zap (str) o numerze k (int) przy użyciu połączenia z bazą w obiekcie conn_.
    Wypisuje czas wykonania zapytania w sekundach (dokładnie tzw. Wall Clock Time, czyli czas rzeczywisty, który upłynął).
    Zwraca wynik zapytania jako ramkę danych Pandas.
    """
    t0 = time.time()  # zapisanie obecnego czasu do zmiennej
    wynik = pd.read_sql_query(zap, conn_)  # wykonanie zapytania
    t = time.time() - t0  # czas (w sekundach), jaki zajęło wykonania zapytania
    print(f'Wykonano zapytanie {k} w {t:.2f}s.')
    return wynik

# Poniższy blok with wykonuje wszystkie 5 zapytań;
# Wyniki umieszcza w zmiennych sql_i.
with sqlite3.connect(SCIEZKA_BAZY) as conn:
    sql_1 = wykonaj_zapytanie(zapytanie_1, 1, conn)
    sql_2 = wykonaj_zapytanie(zapytanie_2, 2, conn)
    sql_3 = wykonaj_zapytanie(zapytanie_3, 3, conn)
    sql_4 = wykonaj_zapytanie(zapytanie_4, 4, conn)
    sql_5 = wykonaj_zapytanie(zapytanie_5, 5, conn)

Wykonano zapytanie 1 w 0.06s.
Wykonano zapytanie 2 w 0.55s.
Wykonano zapytanie 3 w 0.60s.
Wykonano zapytanie 4 w 0.77s.
Wykonano zapytanie 5 w 0.44s.


Moduł `pickle` to system serializacji (konwersji danych do postaci mogącej być zapisaną na dysku/przesłaną przez sieć) obiektów Pythona.
W przeciwieństwie do plików np `csv` można zapisywać niemal dowonly obiekt Pythona. Minusem jest niemożliwość przeczytania takiego obiektu inaczej, niż przez program napisany w Pythonie.

Poniższy kod zapisze wyniki zapytań (ramiki danych Pandas) do takich plików, aby można je potem było szybko odczytać.
**Nie trzeba z niego korzystać, jeżeli nie ma problemów z wykonaniem zapytań SQL.**

In [5]:
# Zapisanie każdej z ramek danych opisujących wyniki zapytań SQL do osobnego pliku pickle.
for i, df in enumerate([sql_1, sql_2, sql_3, sql_4, sql_5], 1):
    df.to_pickle(f'sql_{i}.pkl.gz')

In [6]:
# Wczytanie policzonych uprzednio wyników z plików pickle (możesz to zrobić, jeżeli zapytania wykonują się za długo).
sql_1, sql_2, sql_3, sql_4, sql_5 = [
    pd.read_pickle(f'sql_{i}.pkl.gz') for i in range(1, 5 + 1)
]

## 3. Wyniki zapytań SQL odtworzone przy użyciu metod pakietu Pandas.

Wynikowa ramka danych do zapytania 1 popwinna nazwyać się `pandas_1`, do drugiego `pandas_2` itd.

### Zadanie 1

In [8]:
try:
    pandas_1 = Users[Users.Location != ""]\
    .groupby("Location")\
    ["UpVotes"].sum()\
    .rename("TotalUpVotes")\
    .sort_values(ascending = False)\
    .head(10)\
    .reset_index()


    print(pandas_1.equals(sql_1))

except Exception as e:
    print("Zad. 1: niepoprawny wynik.")
    print(e)

True


In [38]:
pandas_1

,Location,TotalUpVotes
0,"London, UK",26206
1,Netherlands,24146
2,"London, United Kingdom",21941
3,"Oxford, United Kingdom",20158
4,"Christchurch, New Zealand",16352
5,UK,10967
6,"New York, NY",9126
7,Singapore,8905
8,"Cambridge, United Kingdom",8550
9,Australia,8396


In [12]:
pd.read_sql_query(zapytanie_1, conn)

,Location,TotalUpVotes
0,"London, UK",26206
1,Netherlands,24146
2,"London, United Kingdom",21941
3,"Oxford, United Kingdom",20158
4,"Christchurch, New Zealand",16352
5,UK,10967
6,"New York, NY",9126
7,Singapore,8905
8,"Cambridge, United Kingdom",8550
9,Australia,8396


### Zadanie 2

In [32]:
try:
    
    pandas_2 = Posts[Posts.PostTypeId.isin([1, 2])][["CreationDate", "Score"]]
    CreationDate = pd.to_datetime(pandas_2.CreationDate)
    pandas_2["Year"] = CreationDate.dt.strftime("%Y") # dodanie kolumn "Year", "Month"
    pandas_2["Month"] = CreationDate.dt.strftime("%m")
    pandas_2 = pandas_2.groupby(["Year", "Month"])
    MaxScore = pandas_2.Score.max().reset_index(drop = True) # stworzenie nowej ramki danych z max wartosciami 
    pandas_2 = pandas_2.Score.count().reset_index().rename(columns = {"Score": "PostsNumber"})
    pandas_2["MaxScore"] = MaxScore # dolaczenie MaxScore jako kolumny
    pandas_2 = pandas_2[pandas_2.PostsNumber > 1000].reset_index(drop = True)
    print(pandas_2.equals(sql_2))

except Exception as e:
    print("Zad. 2: niepoprawny wynik.")
    print(e)

True


In [36]:
pandas_2.head(10)

,Year,Month,PostsNumber,MaxScore
0,2014,05,1012,133
1,2014,06,1053,74
2,2014,07,1068,80
3,2014,08,1030,58
4,2015,03,1195,73
5,2015,04,1191,71
6,2015,05,1233,90
7,2015,06,1511,165
8,2015,07,1191,81
9,2015,08,1269,126


In [37]:
pd.read_sql_query(zapytanie_2, conn).head(10)

,Year,Month,PostsNumber,MaxScore
0,2014,05,1012,133
1,2014,06,1053,74
2,2014,07,1068,80
3,2014,08,1030,58
4,2015,03,1195,73
5,2015,04,1191,71
6,2015,05,1233,90
7,2015,06,1511,165
8,2015,07,1191,81
9,2015,08,1269,126


### Zadanie 3

In [28]:
try:
    Questions = Posts[Posts.PostTypeId == 1]\
    .groupby("OwnerUserId")\
    ["ViewCount"].sum()\
    .reset_index()\
    .rename(columns = {"ViewCount": "TotalViews"})
    
    pandas_3 = pd.merge(Users, Questions, left_on = "Id", right_on = "OwnerUserId")
    pandas_3 = pandas_3[["Id", "DisplayName", "TotalViews"]]\
    .sort_values(by = "TotalViews", ascending = False)\
    .head(10)\
    .reset_index(drop = True)
    
    print(pandas_3.equals(sql_3))

except Exception as e:
    print("Zad. 3: niepoprawny wynik.")
    print(e)

True


In [29]:
pandas_3

,Id,DisplayName,TotalViews
0,101,Mark Mayo,2151776.0
1,9009,JonathanReez,1941733.0
2,693,RoflcoptrException,1865428.0
3,140,hippietrail,1849585.0
4,26,Gagravarr,1536327.0
5,3736,nsn,1148040.0
6,396,jrdioko,983986.0
7,10051,Nean Der Thal,894454.0
8,583,Jonik,855538.0
9,1893,lambshaanxy,850706.0


In [30]:
pd.read_sql_query(zapytanie_3, conn)

,Id,DisplayName,TotalViews
0,101,Mark Mayo,2151776.0
1,9009,JonathanReez,1941733.0
2,693,RoflcoptrException,1865428.0
3,140,hippietrail,1849585.0
4,26,Gagravarr,1536327.0
5,3736,nsn,1148040.0
6,396,jrdioko,983986.0
7,10051,Nean Der Thal,894454.0
8,583,Jonik,855538.0
9,1893,lambshaanxy,850706.0


### Zadanie 4

In [25]:
try:
    Answers = Posts[Posts.PostTypeId == 2]\
    .groupby("OwnerUserId")["OwnerUserId"]\
    .count()\
    .rename("AnswersNumber")\
    .reset_index()
    
    Questions = Posts[Posts.PostTypeId == 1]\
    .groupby("OwnerUserId")["OwnerUserId"]\
    .count()\
    .rename("QuestionsNumber")\
    .reset_index()
    
    PostsCounts = pd.merge(Answers, Questions, on = "OwnerUserId")
    PostsCounts = PostsCounts[PostsCounts.AnswersNumber > PostsCounts.QuestionsNumber]\
    .sort_values(by = "AnswersNumber", ascending = False)\
    .head(5)\
    .reset_index()
    
    pandas_4 = pd.merge(PostsCounts, Users, left_on = "OwnerUserId", right_on = "Id")
    pandas_4 = pandas_4[["DisplayName", "QuestionsNumber", "AnswersNumber", "Location",
                  "Reputation", "UpVotes", "DownVotes"]]
    
    print(pandas_4.equals(sql_4))
    
except Exception as e:
    print("Zad. 4: niepoprawny wynik.")
    print(e)

True


In [26]:
pandas_4

,DisplayName,QuestionsNumber,AnswersNumber,Location,Reputation,UpVotes,DownVotes
0,Mark Mayo,342,1950,"Christchurch, New Zealand",153872,15817,1613
1,phoog,12,1602,"New York, NY",111746,8212,529
2,Relaxed,8,1414,NaN,91865,10911,570
3,lambshaanxy,104,1394,NaN,99487,13629,253
4,Crazydre,171,1163,NaN,72332,2259,132


In [27]:
pd.read_sql_query(zapytanie_4, conn)

,DisplayName,QuestionsNumber,AnswersNumber,Location,Reputation,UpVotes,DownVotes
0,Mark Mayo,342,1950,"Christchurch, New Zealand",153872,15817,1613
1,phoog,12,1602,"New York, NY",111746,8212,529
2,Relaxed,8,1414,None,91865,10911,570
3,lambshaanxy,104,1394,None,99487,13629,253
4,Crazydre,171,1163,None,72332,2259,132


### Zadanie 5

In [18]:
try:
    CmtTotScr = Comments.groupby(Comments.PostId)["Score"].sum()\
    .rename("CommentsTotalScore")\
    .reset_index()
    
    PostsBestComments = Posts[Posts.PostTypeId == 1]\
    .merge(CmtTotScr, left_on = "Id", right_on = "PostId")

    
    pandas_5 = pd.merge(PostsBestComments, Users, left_on = "OwnerUserId", right_on = "Id")
    pandas_5 = pandas_5[["Title", "CommentCount", "ViewCount", "CommentsTotalScore",
                         "DisplayName", "Reputation", "Location"]]\
    .sort_values(by = "CommentsTotalScore", ascending = False)\
    .head(10)\
    .reset_index(drop = True)
    
    print(pandas_5.equals(sql_5))

except Exception as e:
    print("Zad. 5: niepoprawny wynik.")
    print(e)

True


In [22]:
pandas_5

,Title,CommentCount,ViewCount,CommentsTotalScore,DisplayName,Reputation,Location
0,"Boss is asking for passport, but it has a stam...",24,60532.0,623,Megha,1639,None
1,How to intentionally get denied entry to the U...,37,71829.0,601,user11743,1947,None
2,Can I wear a bulletproof vest while traveling ...,26,22958.0,401,Ulkoma,8413,"London, United Kingdom"
3,"OK we're all adults here, so really, how on ea...",29,107591.0,380,hippietrail,74614,"Sunshine Coast QLD, Australia"
4,Being separated from one's young children on a...,36,13974.0,358,Emma-louise,611,None
5,Immigration officer that stopped me at the air...,24,21808.0,345,DiegoJ,1243,Buenos Aires
6,How to avoid toddlers on a long-distance plane...,18,28968.0,342,Silver Dragon,933,"San Francisco, United States"
7,How to book a flight if my passport doesn't st...,23,17614.0,329,kukis,894,"Sztokholm, Szwecja"
8,Why don't airlines have backup planes just in ...,26,18365.0,328,clickbait,532,None
9,Can I fly with a gold bar?,25,53252.0,322,Ulkoma,8413,"London, United Kingdom"


In [23]:
pd.read_sql_query(zapytanie_5, conn)

,Title,CommentCount,ViewCount,CommentsTotalScore,DisplayName,Reputation,Location
0,"Boss is asking for passport, but it has a stam...",24,60532.0,623,Megha,1639,None
1,How to intentionally get denied entry to the U...,37,71829.0,601,user11743,1947,None
2,Can I wear a bulletproof vest while traveling ...,26,22958.0,401,Ulkoma,8413,"London, United Kingdom"
3,"OK we're all adults here, so really, how on ea...",29,107591.0,380,hippietrail,74614,"Sunshine Coast QLD, Australia"
4,Being separated from one's young children on a...,36,13974.0,358,Emma-louise,611,None
5,Immigration officer that stopped me at the air...,24,21808.0,345,DiegoJ,1243,Buenos Aires
6,How to avoid toddlers on a long-distance plane...,18,28968.0,342,Silver Dragon,933,"San Francisco, United States"
7,How to book a flight if my passport doesn't st...,23,17614.0,329,kukis,894,"Sztokholm, Szwecja"
8,Why don't airlines have backup planes just in ...,26,18365.0,328,clickbait,532,None
9,Can I fly with a gold bar?,25,53252.0,322,Ulkoma,8413,"London, United Kingdom"
